---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    # Your Code Here
    df_movies = pd.read_csv('Employee_Movie_Choices.txt',sep='\t')
    #print(df_movies)
    g = nx.from_pandas_dataframe(df_movies, '#Employee', 'Movie')
    #g = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter="\t")
    return g# Your Answer Here
answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    # Your Code Here
    g = answer_one()
    for node in g.nodes():
        if node in employees:
            g.add_node(node, type="employee")
        else:
            g.add_node(node, type="movie")
    return g# Your Answer Here
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    # Your Code Here
    g=answer_two()
    proyected_g = bipartite.weighted_projected_graph(g, employees)
    return proyected_g# Your Answer Here

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [54]:
import re
def answer_four():
        
    # Your Code Here
    g=answer_three()
    #print(g.edges(data=True))
    df = pd.DataFrame(g.edges(data=True))
    df['weight'] =  df[2].apply(lambda x: re.sub(r'([\'{}a-zA-Z:])','', str(x)))
    df.columns=['1','2','weight','w']
    #print(df)
    
    df_score = pd.read_csv('Employee_Relationships.txt',sep='\t')

    #df_score['corr'] = df_score.corr()
    df_score.columns=['1','2','rel']
    df2 = df_score.append(pd.Series(['Andy','Claude','0',''],index=['1','2','rel','']),ignore_index=True)
    
    #print(df2)
    
    df3 = pd.merge(df, df2, left_on=['1', '2'],right_on=['1', '2'],how='outer')
    df3=df3[['1','2','w','rel']]
    
    df4 = pd.merge(df3, df3, left_on=['1', '2'],right_on=['2', '1'],how='outer')
    df4 = df4.fillna('')
    df4['rel'] = df4['rel_x'].map(str) + df4['rel_y'].map(str)
   
    result = df4[['1_x','2_x','w_x','rel']][:35]
    
    result.columns=['1','2','w','rel']
 
    
    #print(result)
    indexes=[]
    
    for a1, a2 in zip(result['1'][:10],result['2'][:10]):
        for i,b1,b2 in zip(result.index.tolist(),result['1'],result['2']):
            if a1==b2 and a2==b1:
                indexes.append(i)
                    
    result = result.drop(indexes)
    result = result.drop([23])
    
    result['w'][11:]=0
    result = result[:28]
    #print(len(result))
    data = result[['w','rel']]
    data['w'] = data['w'].astype(float)
    data['rel'] = data['rel'].astype(float)
    correlation = data['w'].corr(data['rel'])
    #print(len(result))
    
        
    Rel = nx.read_edgelist('Employee_Relationships.txt', data=[('relationship_score', int)])
    Rel_df = pd.DataFrame(Rel.edges(data=True), columns=['From', 'To', 'relationship_score'])
#     print (Rel_df)
    G = answer_three()
    G_df = pd.DataFrame(G.edges(data=True), columns=['From', 'To', 'movies_score'])
#     print (G_df)
    G_copy_df = G_df.copy()
#     change the edge direction and get a double direction graph
    G_copy_df.rename(columns={"From":"From_", "To":"From"}, inplace=True)
    G_copy_df.rename(columns={"From_":"To"}, inplace=True)
#     print (G_copy_df)
    G_final_df = pd.concat([G_df, G_copy_df])
#     print (G_final_df)
    final_df = pd.merge(G_final_df, Rel_df, on = ['From', 'To'], how='right')
    final_df['movies_score'] = final_df['movies_score'].map(lambda x: x['weight'] if type(x)==dict else None)
    final_df['relationship_score'] = final_df['relationship_score'].map(lambda x: x['relationship_score'])
    final_df['movies_score'].fillna(value=0, inplace=True)
    #print (len(final_df))
    
    #print(correlation)
    #print(final_df['movies_score'].corr(final_df['relationship_score'])-correlation)
    return  final_df['movies_score'].corr(final_df['relationship_score'])# data['w'].corr(data['rel'])# Your Answer Here
answer_four()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.78839622217334737